In [1]:
# 型アノテーション
from typing import Iterator, List, Dict

In [2]:
# AllenNLPはPyTorch上に構築されている
import torch
import torch.optim as optim
import numpy as np

In [3]:
# AllenNLPでは，各トレーニングの例をインスタンスとして表現する
# ここの例では，センテンスを含むTextFieldと対応する品詞を含むSequenceLabelFieldがある
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField

In [4]:
# AllenNLPを使って，ここにあるような問題を解くには，二つのクラスを実装する必要がある
# 1つは，DatasetReaderで，データのファイルを読み込むロジックを含んでおり，インスタンスのストリームを生成する     ★実装しないといけないクラスの１つ目
from allennlp.data.dataset_readers import DatasetReader

In [5]:
# 多くの場合は，URLからデータセットまたはモデルをロードする
# cached_pathヘルパーは，そのようなファイルをダウンロードして，ロカールにキャッシュし，ローカルパスを返す
# ローカルファイルパス（そのまま返される）も使える
from allennlp.common.file_utils import cached_path

In [6]:
# 単語をひとつ以上のインデックスで表現する方法はいろいろある
# 例えば，一意の単語の語彙を維持し，各単語に対応するIDを与えることができる
# または，一文字ごとにひとつのIDを割り当てて，各単語を一連のIDとして表すことができる
# AllenNLPはこの表現に対して，TokenIndexer抽象化を提供している
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token

In [7]:
# TokenIndexerはトークンをインデックスにどのように変換するかのルールを表す
# Vocabularyは文字列から整数への対応するマッピングが含まれている
# 例えば，TokenIndexerはトークンをcharacter IDのシーケンスとして表すように指定できる
# その場合，Vocabularyには，マッピング｛character -> id｝が含まれる
# この特定の例では，各トークンに一意のIDを割り当てるSingledIdTokenIndexerを使用しているので，Vocabularyにはマッピング｛character -> id｝（およびその逆のマッピング）のみが含二つめ
from allennlp.data.vocabulary import Vocabulary

In [8]:
# DatasetReaderのほかに，実装する必要があるクラスはModelです        ★実装しないといけないクラスの２つ目
# これは，テンソル入力を取り，テンソル出力のdictを生成するPyTorchモジュール
from allennlp.models import Model

In [9]:
# モデルは埋め込みレイヤ，それからLSTMに続いて，フィードフォワードレイヤに続く
# AllenNLPは，パディングとバッチ処理，およびさまざまなユーティリティ関数をスマートに処理するこれらすべての抽象化が含まれている
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits

In [10]:
# トレーニングと検証のデータセットの精度を追跡する必要がある
from allennlp.training.metrics import CategoricalAccuracy

In [11]:
# トレーニングでは，データをインテリジェントにバッチ処理できるDataIteratorsが必要
from allennlp.data.iterators import BucketIterator

In [12]:
# そして，AllenNLPのフル機能のトレーナを使う
from allennlp.training.trainer import Trainer

In [13]:
# 最後に新しい入力について予測する
# 詳しくは以下で説明する
from allennlp.predictors import SentenceTaggerPredictor

torch.manual_seed(1)

In [14]:
# まずやることは，DatasetReaderサブクラスを実装すること
class PosDatasetReader(DatasetReader):
    """
    DatasetReader for PoS tagging data, one sentence per line, like

        The###DET dog###NN ate###V the###DET apple###NN
    """

    # DatasetReaderが必要とする唯一のパラメータは，トークンをインデックスに変換する方法を指定するTokenIndexersのdict
    # デフォルトでは，各トークンごとに単一のインデックスを生成する
    # これは，個別のトークンごとの一意のID（これは，ほとんどのNLPタスクで使用する標準の「ワードからインデックス」へのマッピング）
    def __init__(self, token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}

    # DatasetReader.text_to_instanceは，トレーニングの例（この場合は文のトークンと対応する品詞タグ）に対応する入力を受け取り，
    # 対応するField（この場合は文のTextFieldとタグのSequenceLabelField）をインスタンス化する
    # ラベル付けされていないデータからインスタンスを作成して，それらを予測できるようにするため，タグはオプションである
    # （品詞タグがターゲット変数だけど，その答えをすでに持ってるから割り当てておく？みたいな感じ？答え合わせ用？？）
    def text_to_instance(self, tokens: List[Token], tags: List[str] = None) -> Instance:
        self.sentence_field = TextField(tokens, self.token_indexers)
        self.fields = {"sentence": self.sentence_field}

        if tags:
            self.label_field = SequenceLabelField(labels=tags, sequence_field=self.sentence_field)
            self.fields["labels"] = self.label_field

        return Instance(self.fields)
    
    # もう一つ実装する必要があるのは，_readである
    # これは，ファイル名を受け取り，インスタンスのストリームを作成する
    # （このクラスの作業のほとんどは text_to_instance で行われている）
    def _read(self, file_path: str) -> Iterator[Instance]:
        with open(file_path) as f:
            for line in f:
                pairs = line.strip().split()
                sentence, tags = zip(*(pair.split("###") for pair in pairs))
                yield self.text_to_instance([Token(word) for word in sentence], tags)

In [15]:
# 基本的に，常に実装しなければならないもう1つのクラスはModel（これは，torch.nn.Moduleのサブクラス）
# どのように機能するかは主に実装者次第であり，ほとんどの場合，テンソル入力を取り，モデルのトレーニングに使用する損失を含むテンソル出力のdictを生成するforwardメソッドが必要
# 上記のように，Modelは，埋め込みレイヤー，シーケンスエンコーダー，フィードフォワードネットワークで構成される
class LstmTagger(Model):

    # コンストラクタのパラメータとして，
    # 埋め込みパラメータとシーケンスエンコーダーを渡す
    def __init__(self,
                 # 埋め込みレイヤは，トークンをテンソルに変換する一般的な方法を表すAllenNLP TextFieldEmbedderとして指定される
                 # （ここでは，学習したテンソルで各一意の単語を表現したいことを知っているが，一般クラスを使用すると，たとえばELMoなどのさまざまな種類の埋め込みを簡単に試すことができる
                 word_embeddings: TextFieldEmbedder,
                 # 同様に，LSTMを使用したい場合でも，エンコーダーは一般的なSeq2SeqEncoderとして指定される
                 # これにより，トランスフォーマーなどの他のシーケンスエンコーダーを簡単に試すことができる
                 encoder: Seq2SeqEncoder,
                 # すべてのAllenNLPモデルは，トークンからインデックスへのラベルおよびインデックスからラベルへの名前空間マッピングを含むボキャブラリも必要
                 vocab: Vocabulary) -> None:
        # 基本クラスのコンストラクタに語彙を渡す
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder

        # フィードフォワードレイヤはパラメータとして渡されない
        # 正しい入力次元を見つけるためにエンコーダを調べて，正しい出力次元を見つけるために語彙（特に，ラベル→インデックスマッピング）を調べることに注意）
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=vocab.get_vocab_size('labels'))

        # 最後の注意点として，CategoricalAccuracyメトリックをインスタンス化すること
        # これは，各トレーニングおよび検証エポック中に精度を追跡するために使用する
        self.accuracy = CategoricalAccuracy()

    # 次に，実際の計算が行われるフォワードを実装する
    # データセットの各インスタンスは（他のインスタンスとバッチ処理されて）forwardに渡される
    # forwardメソッドは入力としてテンソルのdictを期待し，それらの名前がインスタンス内のフィールドの名前であることを期待する
    # この場合，sentenceフィールドとlabelフィールドがあるので，それに応じてforwardを構築する
    def forward(self,
                sentence: Dict[str, torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str, torch.Tensor]:

        # AllenNLPはバッチ入力で動作するように設計されているが，入力シーケンスごとに長さが異なる．
        # 裏では，AllenNLPが短い入力をパディングして，バッチが同じ形になるようにする
        # つまり，パディングを除外するために，計算ではマスクを使う必要がある
        # これは，パディングされた場所とされていない場所をに対応する0と1のテンソルを返す
        self.mask = get_text_field_mask(sentence)

        # まず，センテンス・テンソル（各センテンスはトークンIDのシーケンス）をword_embeddingsモジュールに渡す
        # word_embeddingsモジュールは，各センテンスを埋め込みテンソルのシーケンスに変換する
        self.embeddings = self.word_embeddings(sentence)

        # 次に，埋め込みテンソル（およびマスク）をLSTMに渡す
        # LSTMは，エンコードされた出力のシーケンスを生成する
        self.encoder_out = self.encoder(self.embeddings, self.mask)

        # 最後に，エンコードされた各出力テンソルをフィードフォワードレイヤに渡して，さまざまなタグに対応する logit を生成する
        self.tag_logits = self.hidden2tag(self.encoder_out)
        self.output = {"tag_logits": self.tag_logits}

        # このモデルを実行してラベル無しデータを予測したい場合があるので，ラベルはオプションである
        # ラベルがある場合は，それらを使用して精度メトリックを更新し，出力に含まれる「損失」を計算する
        if labels is not None:
            self.accuracy(self.tag_logits, labels, self.mask)
            self.output["loss"] = sequence_cross_entropy_with_logits(self.tag_logits, labels, self.mask)

        return self.output

    # フォワードパスごとに更新される精度メトリックを含んでいるので，そのデータを取り出す get_metrics メソッドをオーバーライドする必要がある
    # 裏では，CategoricalAccuracyメトリックは予測の数と正しい予測の数を格納し，転送する各呼び出し中にこれらのカウントを更新する
    # get_metricを呼び出すたびに，計算された精度が返され，（オプションで）カウントがリセットされる
    # これにより，各エポックの精度を新たに追跡できる
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}

In [16]:
# DatasetReaderとModelを実装したので，トレーニングの準備ができた

# まず，DatasetReaderのインスタンスを作成する
reader = PosDatasetReader()

# トレーニングデータと検証データの読み込みに使用できる
# ここでは，URLから読み取りますが，データがローカルの場合はローカルファイルから読み取ることもできる
# 我々は cached_path を使用して，ファイルをローカルにキャッシュする（そして，ローカルのキャッシュバージョンへのパスをreader.readに渡す）
train_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/training.txt'))
validation_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/validation.txt'))

# データセットを読み込んだら，それらを使用してボキャブラリ（トークン／ラベルからIDへのマッピング）を作成する
vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

# 次に，モデルを構築する必要がある
# LSTMの埋め込み層と隠れ層のサイズを選択
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

# トークンを埋め込むには，インデックス名から埋め込みへのマッピングを取る BasicTextFieldEmbedder を使用する
# DatasetReaderを定義した場所に戻ると，デフォルトのパラメータには「トークン」と呼ばれる単一のインデックスが含まれていたため，マッピングにはそのインデックスに対応する埋め込みが必要
# Vocabulary を使用して必要な埋め込みの数を見つけて，EMBEDDING_DIMパラメータを使用して出力の次元を指定する
# 事前学習済みの埋め込み（GloVeベクトルなど）から始めることもできるが，この小さなトイ・データセットでこれを行う必要は無い
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

# 次に，シーケンスエンコーダを指定する必要がある
# ここでのPytorchSeq2SeqWrapperが必要な点は少し残念（設定ファイルを使用する場合は気にしなくていい　→　？）
# ここでは，組み込みのPyTorchモジュールにいくつかの機能を追加する必要がある
# AllenNLPでは，すべてを最初にバッチ処理するので，それも指定する
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

# これで，モデルをインスタンス化できる
model = LstmTagger(word_embeddings, lstm, vocab)

# GPUにアクセスできるか確認する
if torch.cuda.is_available():
    # アクセスできるなら，モデルをGPU 0に移す
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    # アクセスできないなら，modelはそのまま
    cuda_device = -1

2it [00:00, 1969.62it/s]
2it [00:00, 3580.29it/s]
100%|██████████| 4/4 [00:00<00:00, 4030.08it/s]


In [17]:
# このモデルを学習する準備ができた

# 最初に必要なのはオプティマイザである
# PyTorchの確率的勾配降下法を使用できる
optimizer = optim.SGD(model.parameters(), lr=0.1)

# そして，データセットのバッチ処理を行うDataIteratorが必要
# BucketIteratorは，類似したシーケンス長のバッチを作成するために，指定されたフィールドでインスタンスをソートする
# ここでは，センテンスフィールドのトークンの数でインスタンスをソートしている
iterator = BucketIterator(batch_size=2, sorting_keys=[("sentence", "num_tokens")])

# iteratorは，そのインスタンスがボキャブラリを使用してインデックス付けされていることを確認する必要があることも指定する
# つまり，それらの文字列は，以前に作成したマッピングを使用して整数に変換されている
iterator.index_with(vocab)

# ここで，トレーナーをインスタンス化する
# 1000エポックまで実行し，10エポックの間検証メトリックが改善しなければ学習を停止する
# デフォルトの検証メトリックは loss である（小さくなれば改善）が，別のメトリックと方向も指定できる（精度がよくなる，など）
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=1000,
                  cuda_device=cuda_device)

# trainerを起動すると，「損失」と「精度」の両方のメトリックを含む各エポックの進行状況バーが表示される
# このモデルが良ければ，学習中に損失が減少し，精度が上がるはず
trainer.train()

# 元のPyTorchのチュートリアルと同様に，モデルが成功する予測を確認する
# AllenNLPには，入力を受け取り，それらをインスタンスに変換し，モデルを介してフィードし，JSONシリアル化可能な結果を返すPredictor抽象化が含まれている
# 多くの場合，独自の予測子を実装する必要があるが，AllenNLPにはすでにここで完全に機能するSentenceTaggerPredictorがあるので，それを使用できる
# 予測を行うためのModelとインスタンスを生成するためのreaderが必要
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

# predictorはセンテンスが必要なだけのpredictメソッドを持っており，これがforwardからdictを返す
# ここで，tag_logitsは(5, 3)配列で，5ワードのそれぞれの3つの3つの可能なタグに対応している
tag_logits = predictor.predict("The dog ate the apple")['tag_logits']

# 実際の予測を取得するために，argmaxを使う
tag_ids = np.argmax(tag_logits, axis=-1)

# ボキャブラリを使用して予測タグを見つける
print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])

# 最後にモデルを保存して，後でロードできるようにする
# 2つのものを保存する必要がある．
# 1つ目は，モデルの重み
with open("/tmp/model.th", 'wb') as f:
    torch.save(model.state_dict(), f)
# 2つ目は，ボキャブラリ
vocab.save_to_files("/tmp/vocabulary")

# モデルの重みを保存したので，再利用する場合は，実際にコードを使用して同じモデル構造を再作成する必要がある
# まず，ボキャブラリを新しい変数にリロードする
vocab2 = Vocabulary.from_files("/tmp/vocabulary")

# 次に，モデルを再作成する（別のファイルでこれをやろうとしたら，embeddingsとlstmも再インスタンス化が必要）
model2 = LstmTagger(word_embeddings, lstm, vocab2)

# そして，モデルの状態をファイルから読み込む
with open("/tmp/model.th", 'rb') as f:
    model2.load_state_dict(torch.load(f))

# ここで，ロードしたモデルを以前に使用したGPUに移動させる
# 以前に元のモデルでword_embeddingsとlstmを移動したので，これが必要
# モデルのパラメータはすべて同じデバイス上にある必要がある　→　別のデバイスではダメってこと？
if cuda_device > -1:
    model2.cuda(cuda_device)

# そして，以下を実行すると同じ予測を得られます
predictor2 = SentenceTaggerPredictor(model2, dataset_reader=reader)
tag_logits2 = predictor2.predict("The dog ate the apple")['tag_logits']
np.testing.assert_array_almost_equal(tag_logits2, tag_logits)

0:00<00:00, 95.80it/s]
accuracy: 1.0000, loss: 0.0238 ||: 100%|██████████| 1/1 [00:00<00:00, 226.49it/s]
accuracy: 1.0000, loss: 0.0239 ||: 100%|██████████| 1/1 [00:00<00:00, 96.29it/s]
accuracy: 1.0000, loss: 0.0238 ||: 100%|██████████| 1/1 [00:00<00:00, 224.99it/s]
accuracy: 1.0000, loss: 0.0238 ||: 100%|██████████| 1/1 [00:00<00:00, 95.46it/s]
accuracy: 1.0000, loss: 0.0237 ||: 100%|██████████| 1/1 [00:00<00:00, 275.00it/s]
accuracy: 1.0000, loss: 0.0237 ||: 100%|██████████| 1/1 [00:00<00:00, 95.56it/s]
accuracy: 1.0000, loss: 0.0237 ||: 100%|██████████| 1/1 [00:00<00:00, 223.26it/s]
accuracy: 1.0000, loss: 0.0237 ||: 100%|██████████| 1/1 [00:00<00:00, 92.09it/s]
accuracy: 1.0000, loss: 0.0236 ||: 100%|██████████| 1/1 [00:00<00:00, 212.51it/s]
accuracy: 1.0000, loss: 0.0236 ||: 100%|██████████| 1/1 [00:00<00:00, 96.20it/s]
accuracy: 1.0000, loss: 0.0235 ||: 100%|██████████| 1/1 [00:00<00:00, 307.03it/s]
accuracy: 1.0000, loss: 0.0236 ||: 100%|██████████| 1/1 [00:00<00:00, 95.87it/s]

In [18]:
%whos

Variable                             Type                       Data/Info
-------------------------------------------------------------------------
BasicTextFieldEmbedder               type                       <class 'allennlp.modules.<...>.BasicTextFieldEmbedder'>
BucketIterator                       type                       <class 'allennlp.data.ite<...>iterator.BucketIterator'>
CategoricalAccuracy                  type                       <class 'allennlp.training<...>acy.CategoricalAccuracy'>
DatasetReader                        type                       <class 'allennlp.data.dat<...>et_reader.DatasetReader'>
Dict                                 GenericMeta                typing.Dict
EMBEDDING_DIM                        int                        6
Embedding                            type                       <class 'allennlp.modules.<...>ers.embedding.Embedding'>
HIDDEN_DIM                           int                        6
Instance                             Generic